In [1]:
import os, sys, warnings, time, memory_profiler
import numpy as np, matplotlib.pyplot as plt
from sequentia import *
from tqdm.auto import tqdm
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Dropout, LSTM, Bidirectional, Flatten

# Silence TensorFlow warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Import utility functions and classes
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from utils import *
from nn import NNClassifier

# Filter warnings
warnings.filterwarnings('ignore')

# ggplot style
plt.style.use('ggplot')

# Set seed for reproducible randomness
seed = 0
np.random.seed(seed)
rng = np.random.RandomState(seed=seed)

In [2]:
gestures_map = {
    'nd': 'nod',
    'mnd': 'multiple nods',
    'fu': 'face-up',
    'fd': 'face-down',
    'sh': 'shake',
    't': 'turn',
    'ti': 'tilt'
}

gestures = list(gestures_map.keys())

In [3]:
fields = ['Rx', 'Ry', 'Rz']

In [4]:
# Containers to store classifiers and results
clfs, results = {}, {'hmm': {}, 'knn': {}, 'ffnn': {}, 'lstm': {}}

In [5]:
# Load the MoCap dataset
loader = MoCapLoader(normalized=False)
X, y = loader.load(fields)

## Dataset splits

Create a stratified 65-20-15 training, validation and test set split.

## Preprocessing

## DTWKNN classifier

### Fitting the model

### Evaluating the model

## Hidden Markov Model classifier

One gesture model $\lambda_i=(A_i,B_i,\pi_i)$ is initialized and trained for each of the gestures: `nd`, `mnd`, `sh`, `fd`, `t`, `ti`, `fu`.

### Training the model

### Evaluating the model

## Feedforward Neural Network classifier

### Fitting the model

### Evaluating the model

## LSTM classifier

### Fitting the model

### Evaluating the model

# Hyper-parameter searches

## HMM

## DTWKNN

### Radius parameter

### Neighborhood parameter ($k$)

## FFNN

## LSTM

# Test results

## DTWKNN

## HMM

## FFNN

## LSTM

# Preprocessing experiments

## Filtering

## Downsampling (standard)

In [6]:
for repeat in range(5):
    seed = repeat
    np.random.seed(seed)
    rng = np.random.RandomState(seed=seed)
    
    X_train, X_val, X_test, y_train, y_val, y_test = data_split(X, y, (65, 20, 15), random_state=rng, stratify=True)
    
    for factor in (1, 5, 10, 15, 20):
        pre = Preprocess([
            Filter(window_size=10, method='median'),
            Downsample(factor=factor, method='decimate'),
            Center()
        ])

        Xp_train = pre.fit_transform(X_train, verbose=False)
        Xp_val, Xp_test = pre.transform(X_val, verbose=False), pre.transform(X_test, verbose=False)
        
        running_stats = {
            'hmm': {'time_fit': None, 'time_predict': None}, 
            'ffnn': {'time_fit': None, 'time_predict': None}
        }
        
        # HMM classifier (fit)
        t0 = time.time()
        hmms = []
        for g in tqdm(gestures, desc='Training HMMs'):
            hmm = HMM(label=g, n_states=8, random_state=rng)
            hmm.set_random_initial()
            hmm.set_random_transitions()
            hmm.fit([Xp_train[i] for i, label in enumerate(y_train) if label == g])
            hmms.append(hmm)
        hmm_clf = HMMClassifier()
        hmm_clf.fit(hmms)
        running_stats['hmm']['time_fit'] = time.time() - t0
        
        # HMM classifier (predict)
        t0 = time.time()
        running_stats['hmm']['acc'], _ = hmm_clf.evaluate(Xp_val, y_val, labels=gestures)
        running_stats['hmm']['time_predict'] = time.time() - t0
        
        # FFNN classifier (fit)
        t0 = time.time()
        ffnn = NNClassifier(epochs=114, batch_size=128, optimizer=Adam(learning_rate=0.002), classes=gestures)
        ffnn.fit([
                Flatten(),
                Dense(300, activation='relu'),
                Dropout(0.5),
                Dense(300, activation='relu'),
                Dropout(0.5),
                Dense(len(gestures), activation='softmax')
            ], 
            Xp_train, y_train, validation_data=(Xp_val, y_val), verbose=0, return_history=False, early_stop=False
        )
        running_stats['ffnn']['time_fit'] = time.time() - t0
        
        # FFNN classifier (predict)
        t0 = time.time()
        running_stats['ffnn']['acc'], _ = ffnn.evaluate(Xp_val, y_val)
        running_stats['ffnn']['time_predict'] = time.time() - t0
        
        # Write results
        write_downsample_results(running_stats, '{}decimate'.format(factor), 'validation', number=repeat)

Training set size: 841
Validation set size: 259
Test set size: 195



Training set size: 841
Validation set size: 259
Test set size: 195



Training set size: 841
Validation set size: 259
Test set size: 195



Training set size: 841
Validation set size: 259
Test set size: 195



Training set size: 841
Validation set size: 259
Test set size: 195


## Downsampling (bin)

In [7]:
for repeat in range(5):
    seed = repeat
    np.random.seed(seed)
    rng = np.random.RandomState(seed=seed)
    
    X_train, X_val, X_test, y_train, y_val, y_test = data_split(X, y, (65, 20, 15), random_state=rng, stratify=True)
    
    for bin_size in (25, 50, 75, 100, 200):
        pre = Preprocess([
            Filter(window_size=10, method='median'),
            BinDownsample(bin_size=bin_size, method='decimate'),
            Center()
        ])

        Xp_train = pre.fit_transform(X_train, verbose=False)
        Xp_val, Xp_test = pre.transform(X_val, verbose=False), pre.transform(X_test, verbose=False)
        
        running_stats = {
            'hmm': {'time_fit': None, 'time_predict': None}, 
            'ffnn': {'time_fit': None, 'time_predict': None}
        }
        
        # HMM classifier (fit)
        t0 = time.time()
        hmms = []
        for g in tqdm(gestures, desc='Training HMMs'):
            hmm = HMM(label=g, n_states=8, random_state=rng)
            hmm.set_random_initial()
            hmm.set_random_transitions()
            hmm.fit([Xp_train[i] for i, label in enumerate(y_train) if label == g])
            hmms.append(hmm)
        hmm_clf = HMMClassifier()
        hmm_clf.fit(hmms)
        running_stats['hmm']['time_fit'] = time.time() - t0
        
        # HMM classifier (predict)
        t0 = time.time()
        running_stats['hmm']['acc'], _ = hmm_clf.evaluate(Xp_val, y_val, labels=gestures)
        running_stats['hmm']['time_predict'] = time.time() - t0
        
        # FFNN classifier (fit)
        t0 = time.time()
        ffnn = NNClassifier(epochs=114, batch_size=128, optimizer=Adam(learning_rate=0.002), classes=gestures)
        ffnn.fit([
                Flatten(),
                Dense(300, activation='relu'),
                Dropout(0.5),
                Dense(300, activation='relu'),
                Dropout(0.5),
                Dense(len(gestures), activation='softmax')
            ], 
            Xp_train, y_train, validation_data=(Xp_val, y_val), verbose=0, return_history=False, early_stop=False
        )
        running_stats['ffnn']['time_fit'] = time.time() - t0
        
        # FFNN classifier (predict)
        t0 = time.time()
        running_stats['ffnn']['acc'], _ = ffnn.evaluate(Xp_val, y_val)
        running_stats['ffnn']['time_predict'] = time.time() - t0
        
        # Write results
        write_downsample_results(running_stats, '{}bin'.format(bin_size), 'validation', number=repeat)

Training set size: 841
Validation set size: 259
Test set size: 195



Training set size: 841
Validation set size: 259
Test set size: 195



Training set size: 841
Validation set size: 259
Test set size: 195



Training set size: 841
Validation set size: 259
Test set size: 195



Training set size: 841
Validation set size: 259
Test set size: 195
